## Preprocessing

**Interesting links**

https://medium.com/@cd_24/bertopic-fine-tune-parameters-76c3377016fb

**Import Libraries**

In [294]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from umap import UMAP

from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer



import pandas as pd
import openpyxl

**Inputs**

In [295]:
file_name='data_sport_history_chemistry_film_biology.csv'

**Load data**

In [296]:
data = pd.read_csv('input/' + file_name, encoding='utf-8', delimiter=';')
data.head(5)

,sentences
0,Fotbal je populárním sportem ve většině zemí.
1,Tenisový turnaj Wimbledon je jedním z nejstarš...
2,"Běhání je skválý způsob, jak si udržet kondici."
3,"Hokejový zápas může být velmi napínavý, zejmén..."
4,Plavání je vynikající cvičení pro posílení sva...


**Put data in a list**

In [297]:
docs = [item for sublist in data.values.tolist() for item in sublist]
docs[:5]

['Fotbal je populárním sportem ve většině zemí.',
 'Tenisový turnaj Wimbledon je jedním z nejstarších a nejprestižnějších na světě.',
 'Běhání je skválý způsob, jak si udržet kondici.',
 'Hokejový zápas může být velmi napínavý, zejména v prodloužení.',
 'Plavání je vynikající cvičení pro posílení svalů celého těla.']

In [298]:
len(docs)

499

## Set up model

**Define the clustering model**

In [299]:
hdbscan_model = HDBSCAN(min_cluster_size=30, 
                        min_samples=10,
                        #gen_min_span_tree=True,
                        #prediction_data=True
                        )

In [300]:
#does not produce any outliers!!!
kmeans_model = KMeans(n_clusters=8)

**Define Count Vectorizer**

In [301]:
# Read your custom Czech stop words from the CSV file
stopwords_df = pd.read_csv('input/stop_words_czech.txt')
czech_stop_words = set(stopwords_df['stopwords'].tolist())


# Convert the set of stop words into a list
czech_stop_words_list = list(czech_stop_words)
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=czech_stop_words_list)

**Define the UMAP**


In [302]:
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)

**Define Sentence Transformer**

In [303]:
# model 'all-MiniLM-L6-v2' should be multilingual
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

**Define model**

In [304]:
model = BERTopic(#language="czech", # not necessary if sentence transformer is chosen
                 embedding_model = embedding_model,
                 min_topic_size=10,
                 top_n_words=15,
                 #calculate_probabilities=True, # takes more time - for hdb-scan only
                 #vectorizer_model=vectorizer_model, 
                 hdbscan_model = kmeans_model, 
                 #hdbscan_model = hdbscan_model,
                 umap_model = umap_model
                 )

## Train model

In [305]:
""" It is not going to work with a few hundereds documents, better to have thousands"""

topics, probs = model.fit_transform(docs)

## Coherence score

In [306]:
#https://github.com/MaartenGr/BERTopic/issues/90

import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
import pandas as pd


'''
if error boolean - run the whole notebook again!!!
'''

def coherence_score(model, docs: pd.DataFrame, topics):
    # Merge all sentences for certain topic
    documents = pd.DataFrame({"Document": docs,
                            #"ID": range(len(docs)),
                            "Topic": topics})

    documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
    cleaned_docs = model._preprocess_text(documents_per_topic.Document.values)

    # Extract vectorizer and analyzer from BERTopic
    vectorizer = model.vectorizer_model
    analyzer = vectorizer.build_analyzer()

    # Use '.get_feature_names_out()' if you get an error with '.get_feature_names()'
    words = vectorizer.get_feature_names_out()

    # Extract features for Topic Coherence evaluation
    tokens = [analyzer(doc) for doc in cleaned_docs]
    dictionary = corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(token) for token in tokens]


    # Extract words in each topic if they are non-empty and exist in the dictionary - SIMPLE 
    topic_words = [[words for words, _ in model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

    # Evaluate Coherence
    coherence_model = CoherenceModel(topics=topic_words, 
                                    texts=tokens, 
                                    corpus=corpus,
                                    dictionary=dictionary, 
                                    coherence='c_v')
    coherence = coherence_model.get_coherence()
    return round(coherence,3)

In [307]:
coherence_score(model=model, docs=docs, topics=topics)

0.771

## Visualize results

In [308]:
model.get_topic_freq()

,Topic,Count
2,0,124
0,1,86
3,2,83
5,3,77
1,4,65
4,5,26
6,6,24
7,7,14


In [309]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,124,0_roce_byl_po_století,"[roce, byl, po, století, na, první, ve, české,...",[Třicetiletá válka v 17. století měla tragický...
1,1,86,1_je_sport_sportem_který,"[je, sport, sportem, který, oblíbeným, na, pro...",[Cyklistika je oblíbeným sportem pro rekreační...
2,2,83,2_film_český_získal_za,"[film, český, získal, za, je, filmy, filmová, ...",[Kolya získal Oscara za nejlepší cizojazyčný f...
3,3,77,3_organismů_jsou_organismy_živých,"[organismů, jsou, organismy, živých, dna, jeji...",[Ekosystémy jsou vzájemně propojené sítě živýc...
4,4,65,4_energii_jsou_fotosyntéza_rychlosti,"[energii, jsou, fotosyntéza, rychlosti, atomy,...","[Fotosyntéza je proces, při kterém rostliny vy..."
5,5,26,5_buněk_je_proces_prostředí,"[buněk, je, proces, prostředí, bílé, univerzit...",[Karlova univerzita v Praze byla založena v ro...
6,6,24,6_reakce_exotermické_chemické_endotermické,"[reakce, exotermické, chemické, endotermické, ...",[Chemické reakce mohou být exotermické nebo en...
7,7,14,7_prvky_periodická_tabulka_vlastností,"[prvky, periodická, tabulka, vlastností, podle...",[Periodická tabulka rozděluje prvky podle jeji...


In [310]:
model.visualize_barchart(top_n_topics=10)

In [311]:
model.visualize_topics()

In [312]:
model.visualize_heatmap(top_n_topics=20, width=800, height=800)

## Representation - Count Vectorizer

**Update topics**

In [313]:
model.update_topics(docs, vectorizer_model=vectorizer_model)

**Get counts of the topics**

In [314]:
results_unique_topics = model.get_topic_info()
results_unique_topics[:20]

,Topic,Count,Name,Representation,Representative_Docs
0,0,124,0_roce_století_české_měla,"[roce, století, české, měla, světové, revoluce...",[Třicetiletá válka v 17. století měla tragický...
1,1,86,1_sport_sportem_oblíbeným_zahrnuje,"[sport, sportem, oblíbeným, zahrnuje, cyklisti...",[Cyklistika je oblíbeným sportem pro rekreační...
2,2,83,2_film_český_získal_filmy,"[film, český, získal, filmy, filmová, filmu, č...",[Kolya získal Oscara za nejlepší cizojazyčný f...
3,3,77,3_organismů_organismy_dna_živých,"[organismů, organismy, dna, živých, systému, ž...",[Ekosystémy jsou vzájemně propojené sítě živýc...
4,4,65,4_energii_fotosyntéza_atomy_rychlosti,"[energii, fotosyntéza, atomy, rychlosti, chemi...","[Fotosyntéza je proces, při kterém rostliny vy..."
5,5,26,5_buněk_proces_prostředí_střední evropě,"[buněk, proces, prostředí, střední evropě, bíl...",[Karlova univerzita v Praze byla založena v ro...
6,6,24,6_reakce_chemické reakce_exotermické_chemické,"[reakce, chemické reakce, exotermické, chemick...",[Chemické reakce mohou být exotermické nebo en...
7,7,14,7_prvky_tabulka_periodická tabulka_periodická,"[prvky, tabulka, periodická tabulka, periodick...",[Periodická tabulka rozděluje prvky podle jeji...


**Coherence score**

In [315]:
coherence_score(model=model, docs=docs, topics=topics)

0.71

**Export voctorized data**

In [316]:
#get results into a df
results_vect = model.get_document_info(docs)

# Export to Excel
results_vect.to_excel('output/BERTopic-results-vectorized.xlsx', index=False, engine='openpyxl')

## Topic reduction

In [317]:
""" # manually merge topics
#https://maartengr.github.io/BERTopic/getting_started/topicreduction/topicreduction.html#manual-topic-reduction

topics_to_merge = [[1, 2]
                   [3, 4]]
topic_model.merge_topics(docs, topics_to_merge) """

' # manually merge topics\n#https://maartengr.github.io/BERTopic/getting_started/topicreduction/topicreduction.html#manual-topic-reduction\n\ntopics_to_merge = [[1, 2]\n                   [3, 4]]\ntopic_model.merge_topics(docs, topics_to_merge) '

In [318]:
# Further reduce topics
new_model = model.reduce_topics(docs, nr_topics=5)

In [319]:
new_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,203,0_roce_století_chemická_české,"[roce, století, chemická, české, energii, měla...","[Fotosyntéza je proces, při kterém rostliny vy..."
1,1,163,1_sport_sportem_organismů_organismy,"[sport, sportem, organismů, organismy, oblíben...","[Atletika zahrnuje běh, skok do dálky a vrh ko..."
2,2,83,2_film_český_získal_filmy,"[film, český, získal, filmy, filmová, filmu, č...",[Kolya získal Oscara za nejlepší cizojazyčný f...
3,3,26,3_buněk_proces_prostředí_střední evropě,"[buněk, proces, prostředí, střední evropě, bíl...",[Karlova univerzita v Praze byla založena v ro...
4,4,24,4_reakce_chemické reakce_exotermické_chemické,"[reakce, chemické reakce, exotermické, chemick...",[Chemické reakce mohou být exotermické nebo en...


In [320]:
new_results=new_model.get_document_info(docs)
new_results.head(100)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document
0,Fotbal je populárním sportem ve většině zemí.,1,1_sport_sportem_organismů_organismy,"[sport, sportem, organismů, organismy, oblíben...","[Atletika zahrnuje běh, skok do dálky a vrh ko...",sport - sportem - organismů - organismy - oblí...,False
1,Tenisový turnaj Wimbledon je jedním z nejstarš...,1,1_sport_sportem_organismů_organismy,"[sport, sportem, organismů, organismy, oblíben...","[Atletika zahrnuje běh, skok do dálky a vrh ko...",sport - sportem - organismů - organismy - oblí...,False
2,"Běhání je skválý způsob, jak si udržet kondici.",1,1_sport_sportem_organismů_organismy,"[sport, sportem, organismů, organismy, oblíben...","[Atletika zahrnuje běh, skok do dálky a vrh ko...",sport - sportem - organismů - organismy - oblí...,False
3,"Hokejový zápas může být velmi napínavý, zejmén...",0,0_roce_století_chemická_české,"[roce, století, chemická, české, energii, měla...","[Fotosyntéza je proces, při kterém rostliny vy...",roce - století - chemická - české - energii - ...,False
4,Plavání je vynikající cvičení pro posílení sva...,1,1_sport_sportem_organismů_organismy,"[sport, sportem, organismů, organismy, oblíben...","[Atletika zahrnuje běh, skok do dálky a vrh ko...",sport - sportem - organismů - organismy - oblí...,False
...,...,...,...,...,...,...,...
95,Chemická analýza může odhalit složení látek a ...,1,1_sport_sportem_organismů_organismy,"[sport, sportem, organismů, organismy, oblíben...","[Atletika zahrnuje běh, skok do dálky a vrh ko...",sport - sportem - organismů - organismy - oblí...,False
96,"Rovnováha v chemii se vyskytuje, když rychlost...",0,0_roce_století_chemická_české,"[roce, století, chemická, české, energii, měla...","[Fotosyntéza je proces, při kterém rostliny vy...",roce - století - chemická - české - energii - ...,False
97,Exotermické reakce uvolňují energii do okolí v...,4,4_reakce_chemické reakce_exotermické_chemické,"[reakce, chemické reakce, exotermické, chemick...",[Chemické reakce mohou být exotermické nebo en...,reakce - chemické reakce - exotermické - chemi...,False
98,Chirální molekuly mají zrcadlově symetrické ve...,0,0_roce_století_chemická_české,"[roce, století, chemická, české, energii, měla...","[Fotosyntéza je proces, při kterém rostliny vy...",roce - století - chemická - české - energii - ...,False


In [321]:
new_results.to_excel('output/BERTopic-results-reduced_topics.xlsx', index=False, engine='openpyxl')

## Translate

In [322]:
""" from deep_translator import GoogleTranslator

def translate_sentences(sentences, source_language='cs', target_language='en'):
    translated_sentences = []
    
    for sentence in sentences:
        translated = GoogleTranslator(source=source_language, target=target_language).translate(sentence)
        translated_sentences.append(translated)
    
    return translated_sentences

# Example usage
sentences_to_translate = [
    'snažím se dovolat uz minut',
    'ahoj jak se mas',
    'mam se dobre a ty?'
]

source_language = 'cs'
target_language = 'en'

translated_sentences = translate_sentences(sentences_to_translate, source_language, target_language)

# Print the original and translated sentences
for original, translated in zip(sentences_to_translate, translated_sentences):
    print(f"Original: {original}")
    print(f"Translated: {translated}")
    print() """

' from deep_translator import GoogleTranslator\n\ndef translate_sentences(sentences, source_language=\'cs\', target_language=\'en\'):\n    translated_sentences = []\n    \n    for sentence in sentences:\n        translated = GoogleTranslator(source=source_language, target=target_language).translate(sentence)\n        translated_sentences.append(translated)\n    \n    return translated_sentences\n\n# Example usage\nsentences_to_translate = [\n    \'snažím se dovolat uz minut\',\n    \'ahoj jak se mas\',\n    \'mam se dobre a ty?\'\n]\n\nsource_language = \'cs\'\ntarget_language = \'en\'\n\ntranslated_sentences = translate_sentences(sentences_to_translate, source_language, target_language)\n\n# Print the original and translated sentences\nfor original, translated in zip(sentences_to_translate, translated_sentences):\n    print(f"Original: {original}")\n    print(f"Translated: {translated}")\n    print() '